In [1]:
import requests
import pandas as pd
import time

# Base URL for the feature service layer (you’ll generally need to pick a layer ID)
BASE_URL = "https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/USA_Wildfires_v1/FeatureServer"
LAYER_ID = 0  # adjust if the wildfire data lives in another layer

def fetch_features_chunk(base_url: str, layer_id: int, offset: int = 0, limit: int = 2000) -> dict:
    """
    Fetch a chunk/page of features from the FeatureServer layer.
    Returns the parsed JSON dict.
    """
    url = f"{base_url}/{layer_id}/query"
    params = {
        "f": "json",
        "where": "1=1",
        "outFields": "*",
        "resultOffset": offset,
        "resultRecordCount": limit,
        # optionally, you can specify an orderBy to keep consistent pagination
        "orderByFields": "OBJECTID ASC",
        # you can turn off geometry if you only want attributes:
        # "returnGeometry": "false"
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    return resp.json()

def fetch_all_features(base_url: str, layer_id: int, chunk_size: int = 2000, pause: float = 0.2) -> list:
    """
    Fetch all features by paging through the dataset.
    Returns a list of feature dicts (with geometry & attributes).
    """
    all_feats = []
    offset = 0

    while True:
        js = fetch_features_chunk(base_url, layer_id, offset=offset, limit=chunk_size)
        feats = js.get("features", [])
        if not feats:
            break
        all_feats.extend(feats)

        # If the number returned is less than the chunk_size, we’re likely done
        if len(feats) < chunk_size:
            break

        offset += chunk_size
        time.sleep(pause)  # be polite / avoid hammering the server

    return all_feats

def features_to_dataframe(features: list, include_geometry: bool = True) -> pd.DataFrame:
    """
    Convert a list of feature dicts (GeoJSON-like with "attributes" and "geometry") to a pandas DataFrame.
    If include_geometry is False, only includes the attributes.
    """
    # Separate attribute dicts and optionally geometries
    attr_list = []
    geom_list = []
    for feat in features:
        attrs = feat.get("attributes", {})
        attr_list.append(attrs)
        if include_geometry:
            geom = feat.get("geometry")
            # you may want to transform or flatten geometry (e.g. x, y, etc.)
            geom_list.append(geom)
    df = pd.DataFrame(attr_list)
    if include_geometry:
        # Attach geometry as a column (you can expand it further)
        df["geometry"] = geom_list
    return df

def load_wildfire_data(base_url: str = BASE_URL, layer_id: int = LAYER_ID) -> pd.DataFrame:
    """
    Fetch all wildfire features and return a pandas DataFrame.
    """
    features = fetch_all_features(base_url, layer_id)
    df = features_to_dataframe(features, include_geometry=True)
    return df

if __name__ == "__main__":
    df = load_wildfire_data()
    print("Loaded rows:", len(df))
    print(df.columns)
    print(df.head())


Loaded rows: 339
Index(['OBJECTID', 'IncidentName', 'IncidentTypeCategory',
       'UniqueFireIdentifier', 'DailyAcres', 'CalculatedAcres',
       'PercentContained', 'ICS209ReportDateTime', 'FireDiscoveryDateTime',
       'DiscoveryAcres', 'POOCounty', 'POOState', 'FireCause',
       'FireCauseGeneral', 'GACC', 'TotalIncidentPersonnel',
       'IncidentManagementOrganization', 'FireMgmtComplexity',
       'ResidencesDestroyed', 'OtherStructuresDestroyed', 'Injuries',
       'Fatalities', 'PredominantFuelGroup', 'PredominantFuelModel',
       'PrimaryFuelModel', 'ContainmentDateTime', 'ControlDateTime',
       'FinalAcres', 'IsValid', 'FireOutDateTime', 'ModifiedOnDateTime',
       'IncidentTypeKind', 'IrwinID', 'GlobalID', 'ModifiedOnAge',
       'FireDiscoveryAge', 'geometry'],
      dtype='object')
   OBJECTID           IncidentName IncidentTypeCategory UniqueFireIdentifier  \
0         1               BEAR DEN                   WF    2024-NDFBA-000069   
1         2           TUFF 

In [11]:
import requests

BASE = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/InFORM_FireOccurrence_Public/FeatureServer/0/query"

def fetch_features(where="1=1", out_fields="*", max_records=1000, offset=0):
    """
    Fetch features from the ArcGIS FeatureServer.
    If there are more than max_records, you can call repeatedly with offset.
    """
    params = {
        "where": where,
        "outFields": out_fields,
        "f": "json",
        "resultOffset": offset,
        "resultRecordCount": max_records,
        # Optionally, you can request geometry if needed:
        # "returnGeometry": "true",
        # "outSR": 4326,
    }
    resp = requests.get(BASE, params=params)
    resp.raise_for_status()
    return resp.json()

def fetch_all(where="1=1", out_fields="*", page_size=1000):
    """
    Iterate / page through results (if total > page_size).
    """
    all_feats = []
    offset = 0
    while True:
        js = fetch_features(where=where, out_fields=out_fields, max_records=page_size, offset=offset)
        feats = js.get("features", [])
        all_feats.extend(feats)
        # If fewer than a full page returned, we’re done
        if len(feats) < page_size:
            break
        offset += page_size
    return all_feats

if __name__ == "__main__":
    #features = fetch_all(where="1=1", out_fields="*")
    features = fetch_features()
    print(f"Got {len(features)} features")
    # Example: print first feature
    print("First features")
    print(features["features"])

Got 8 features
First features
[{'attributes': {'OBJECTID': 2237830, 'ABCDMisc': None, 'ADSPermissionState': 'DEFAULT', 'CalculatedAcres': None, 'ContainmentDateTime': 1624893300000, 'ControlDateTime': None, 'CreatedBySystem': 'BulkUpload', 'CreatedOnDateTime': 1660852075725, 'IncidentSize': 0.1, 'DiscoveryAcres': None, 'DispatchCenterID': 'COGJC', 'EstimatedCostToDate': None, 'FinalAcres': None, 'FinalFireReportApprovedByTitle': None, 'FinalFireReportApprovedByUnit': None, 'FinalFireReportApprovedDate': None, 'FireBehaviorGeneral': None, 'FireCode': None, 'FireDepartmentID': None, 'FireDiscoveryDateTime': 1624892280000, 'FireMgmtComplexity': None, 'FireOutDateTime': None, 'FSJobCode': None, 'FSOverrideCode': None, 'GACC': 'RMCC', 'IncidentName': 'Unknown', 'IncidentShortDescription': None, 'IncidentTypeCategory': 'WF', 'IncidentTypeKind': 'FI', 'InitialLatitude': 39.2444, 'InitialLongitude': -108.9378, 'InitialResponseDateTime': None, 'IsFireCauseInvestigated': None, 'IsFSAssisted': No

KeyError: 0